In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.features.aggregate_features import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

### LFTSV

In [6]:
%%time
# Data ingestion, cleaning/wrangling
df_lftsv_raw = pull_data(config_path, lftsv_sql(), 'DWH', 'lftsv')

,client_mifos_id,client_mobile_number,loan_status,loan_mifos_id,term_frequency,principal_disbursed,principal_repaid,interest_charged,interest_repaid,fee_charges_charged,fee_charges_repaid,penalty_charges_charged,penalty_charges_repaid,total_outstanding,disbursed_on_date,expected_matured_on_date,closed_on_date,store_number,bloom_version
263867,36214,None,600,88095,7,5000.00,5000.00,100.00,100.00,0.00,0.00,0.00,0.00,0.00,2019-06-09,2019-06-16,2019-06-16,772618,1.00
73684,26413,None,600,68940,7,8000.00,8000.00,208.01,208.01,301.81,301.81,7.30,7.30,0.00,2022-04-15,2022-04-22,2022-05-20,7801990,2.00


Wall time: 4min 50s


In [7]:
%%time
# Data ingestion, cleaning/wrangling
df_lftsv_clean = clean_lftsv_data(config_path, df_lftsv_raw)

Analysis start date 2017-12-10 00:00:00
Analysis latest date 2022-09-21 00:00:00


,client_mifos_id,client_mobile_number,loan_status,loan_mifos_id,term_frequency,principal_disbursed,principal_repaid,interest_charged,interest_repaid,fee_charges_charged,fee_charges_repaid,penalty_charges_charged,penalty_charges_repaid,total_outstanding,disbursed_on_date,expected_matured_on_date,closed_on_date,store_number,bloom_version
227837,18086,0,600,4936,21,71800.00,71800.00,6241.88,6241.88,3590.00,3590.00,663.19,663.19,0.00,2021-11-24,2021-12-15,2022-04-03,0,2.00
415299,31288,0,600,228961,30,24700.00,24700.00,1729.00,1729.00,0.00,0.00,0.00,0.00,0.00,2021-10-27,2021-11-26,2021-11-08,0,1.00


Wall time: 5.62 s


In [8]:
%%time
# Feature engineering
df_lftsv_features = lftsv_feature_engineering(config_path, df_lftsv_clean, tday)

,client_mifos_id,client_mobile_number,loan_status,loan_mifos_id,term_frequency,principal_disbursed,principal_repaid,interest_charged,interest_repaid,fee_charges_charged,fee_charges_repaid,penalty_charges_charged,penalty_charges_repaid,total_outstanding,disbursed_on_date,expected_matured_on_date,closed_on_date,store_number,bloom_version,loan_id_product_concat,loan_count,loan_rank,due_date_fixed,days_past_due,loan_repayment_status
323022,41710,0,600,218127,7,5300.00,5300.00,106.00,106.00,257.00,257.00,1066.00,1066.00,0.00,2021-08-05,2021-08-12,2021-09-25,0,1.00,218127-1.0,360369,150367.00,2021-08-12,44.00,closed_default
395030,12113,0,600,171393,30,5000.00,5000.00,350.00,350.00,0.00,0.00,0.00,0.00,0.00,2019-12-11,2020-01-10,2020-01-10,0,1.00,171393-1.0,360369,111967.50,2020-01-10,0.00,closed_on_time


Wall time: 2.64 s


In [9]:
%%time
# Data aggregation
df_lftsv_aggregate = agg_summaries(config_path, df_lftsv_features, tday)

ValueError: Cannot take a larger sample than population when 'replace=False'